In [1]:
from bs4 import BeautifulSoup, Comment
import requests
import regex as re
import xml
from selenium import webdriver
import pandas as pd

#Remove HTML T A G S by regex |||||||||| Actually, use get_text() Function. It's much easier. =,,=

In [2]:
#USD

website = ["https://th.investing.com/currencies/usd-thb-historical-data",
"https://th.investing.com/currencies/jpy-thb-historical-data",
"https://th.investing.com/currencies/cny-thb-historical-data",
]


In [3]:
def remove_tag(html_str):
    TAG_RE = re.compile(r'<[^>]+>')
    text = TAG_RE.sub('!@', html_str)
    mylist = re.split(r'!@', text)
    clear_lst = []
    for clear_text in mylist:
        if (clear_text not in "\n") and (clear_text not in " ") and (clear_text not in ""):
            clear_lst.append(clear_text)
    return clear_lst
#https://www.guru99.com/python-regular-expressions-complete-tutorial.html

In [4]:
def re_find(tag_start,tag_end,text):
    a = re.search(tag_start, text)
    if a == None:
        return ''
    text = text[a.start():]
    b = re.search(tag_end, text)
    text = text[:b.end()]
    #print(a.start(),b.start())
    return text

In [5]:
def re_find_all(tag_start,tag_end,text):
    list_all = []
    while(True):
        a = re.search(tag_start, text)
        #print(a.start(),b.end())
        if(a == None):
            #print("break")
            return list_all
        text = text[a.start():]
        b = re.search(tag_end, text)
        list_all.append(text[:b.end()])
        text=text[b.end():]
    return list_all

#start browser

In [6]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
# executable_path param is not needed if you updated PATH

browser = webdriver.Chrome(options=options, executable_path='99.0.4844.51/chromedriver_win32/chromedriver.exe')

C:\Users\Administrator\AppData\Local\Temp\ipykernel_8292\924491811.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(options=options, executable_path='99.0.4844.51/chromedriver_win32/chromedriver.exe')


In [12]:
browser.get(website[0])
html = browser.page_source
soup = BeautifulSoup(html, features="html.parser")

In [8]:
#data = soup.find('div',{"id" : "results_box"}).find('table',{'class':'genTbl closedTbl historicalTbl'}).find('tbody').find_all('tr')


In [24]:
data = re_find_all(r'<tr>',r'</tr>',re_find(r'<tbody>',r'</tbody>',re_find(r'<table class="genTbl closedTbl historicalTbl" id="curr_table" tablesorter="">',r'</table>',re_find(r'<div id="results_box">',r'</div>',str(soup)))))

In [25]:
data

['<tr>\n<td class="first left bold noWrap" data-real-value="1648166400">Mar 25, 2022</td>\n<td class="greenFont" data-real-value="33.600">33.600</td>\n<td data-real-value="33.530">33.530</td>\n<td data-real-value="33.610">33.610</td>\n<td data-real-value="33.420">33.420</td>\n<td class="bold greenFont">0.27%</td>\n</tr>',
 '<tr>\n<td class="first left bold noWrap" data-real-value="1648080000">Mar 24, 2022</td>\n<td class="redFont" data-real-value="33.510">33.510</td>\n<td data-real-value="33.610">33.610</td>\n<td data-real-value="33.720">33.720</td>\n<td data-real-value="33.505">33.505</td>\n<td class="bold redFont">-0.30%</td>\n</tr>',
 '<tr>\n<td class="first left bold noWrap" data-real-value="1647993600">Mar 23, 2022</td>\n<td class="greenFont" data-real-value="33.610">33.610</td>\n<td data-real-value="33.510">33.510</td>\n<td data-real-value="33.700">33.700</td>\n<td data-real-value="33.445">33.445</td>\n<td class="bold greenFont">0.39%</td>\n</tr>',
 '<tr>\n<td class="first left b

#pack data to csv file

In [26]:
col = ['Date', 'Price', 'Open','High','Low','Change(%)']
df = pd.DataFrame()
df = df.reindex(columns = col)

In [27]:
for i,d in enumerate(data):
    dd = remove_tag(str(d))
    df.loc[len(df)] = dd

In [28]:
df

,Date,Price,Open,High,Low,Change(%)
0,"Mar 25, 2022",33.600,33.530,33.610,33.420,0.27%
1,"Mar 24, 2022",33.510,33.610,33.720,33.505,-0.30%
2,"Mar 23, 2022",33.610,33.510,33.700,33.445,0.39%
3,"Mar 22, 2022",33.480,33.540,33.710,33.450,-0.18%
4,"Mar 21, 2022",33.540,33.280,33.570,33.270,0.75%
5,"Mar 18, 2022",33.290,33.210,33.400,33.195,0.27%
6,"Mar 17, 2022",33.200,33.400,33.400,33.180,-0.42%
7,"Mar 16, 2022",33.340,33.540,33.560,33.250,-0.51%
8,"Mar 15, 2022",33.510,33.450,33.570,33.420,0.18%
9,"Mar 14, 2022",33.450,33.300,33.470,33.270,0.45%


In [37]:
df.to_csv('thb-usd.csv',index=False)

#close browser

In [38]:
browser.quit()

In [13]:
f = open('text2.txt', 'w')
with open('text2.txt', 'w',encoding='utf-8') as f:
    f.write(str(soup))

In [113]:
#soup